# Visualisation tools <img align="right" src="../resources/csiro_easi_logo.png">

Data visualisation helps to tell stories by curating and transforming data into a form that is easier to understand and engage with, removing the noise and highlighting useful information. 
In this notebook we will explore some of the ways you can visualise and represent your data including exploring the different code libraries in EASI, visualisation strategies and their use cases.

References:
- [3 - Data and visualisation python libraries](https://csiroau.sharepoint.com/:w:/r/sites/LearningAcademy-ProjectFundamentals-EASIOnboarding/Shared%20Documents/EASI%20Onboarding%20-%20Collaboration/Module%20content%20and%20resources/Module%20Content/3%20-%20Data%20and%20visualisation%20python%20libraries.docx?d=wda1fadb60bdc40a191d23e7f262f92cf&csf=1&web=1&e=ofbTUn)
- https://datashader.org/getting_started/Pipeline.html
- https://towardsdatascience.com/an-incomplete-guide-to-accessible-data-visualization-33f15bfcc400
- https://www.highcharts.com/blog/tutorials/10-guidelines-for-dataviz-accessibility/
- https://it.wisc.edu/learn/make-it-accessible/accessible-data-visualizations/

### Selection of python visualisation libraries available in EASI

- Plotly
- Folium
- Seaborn

## Imports

In [ ]:
# Common imports and settings
import os, sys
os.environ['USE_PYGEOS'] = '0'
from IPython.display import Markdown
import pandas as pd
pd.set_option("display.max_rows", None)
import xarray as xr

# Datacube
import datacube
from datacube.utils.rio import configure_s3_access
import odc.geo.xr                                  # https://github.com/opendatacube/odc-geo
from datacube.utils import masking  # https://github.com/opendatacube/datacube-core/blob/develop/datacube/utils/masking.py
from odc.algo import enum_to_bool                  # https://github.com/opendatacube/odc-tools/blob/develop/libs/algo/odc/algo/_masking.py
from dea_tools.plotting import display_map, rgb    # https://github.com/GeoscienceAustralia/dea-notebooks/tree/develop/Tools

# Dask
from dask.distributed import Client, LocalCluster

# EASI defaults
import git
repo = git.Repo('.', search_parent_directories=True)
if repo.working_tree_dir not in sys.path: sys.path.append(repo.working_tree_dir)
from easi_tools import EasiDefaults
from easi_tools.notebook_utils import mostcommon_crs, initialize_dask, localcluster_dashboard, heading
from easi_tools.load_s2l2a import load_s2l2a_with_offset

In [ ]:
# EASI defaults
easi = EasiDefaults()

family = 'sentinel-2'
product = easi.product(family)
display(Markdown(f'Default {family} product for "{easi.name}": [{product}]({easi.explorer}/products/{product})'))

In [ ]:
# Local cluster
cluster = LocalCluster(n_workers=2, threads_per_worker=4)
client = Client(cluster)
display(client)

dashboard_address = localcluster_dashboard(client=client,server=easi.hub)
display(f'Dashboard: {dashboard_address}')

### Example data

In [ ]:
# Datacube (Xarray)
dc = datacube.Datacube()
configure_s3_access(aws_unsigned=False, requester_pays=True, client=client);

# Default area of interest
display(Markdown(f'#### Location: {easi.location}'))
display(Markdown(f'See: {easi.explorer}/products/{product}'))

latitude_range = easi.latitude
longitude_range = easi.longitude
time_range = easi.time
# Or set your own latitude / longitude
# latitude_range = (21.5, 23.5)
# longitude_range = (88, 90.8)
# time_range = ('2022-01-01', '2022-03-01')

query = {
    'product': product,       # Product name
    'x': longitude_range,     # "x" axis bounds
    'y': latitude_range,      # "y" axis bounds
    'time': time_range,       # Any parsable date strings
}

# Most common CRS
native_crs = mostcommon_crs(dc, query)
print(f'Most common native CRS: {native_crs}')

# Measurement aliases
alias = easi.aliases(family)
measurements = [alias[x] for x in ['blue', 'red', 'green', 'nir', 'scl']]

# Target xarray parameters
load_params = {
    'measurements': measurements,  # Selected measurement or alias names
    'output_crs': native_crs,                       # Target EPSG code
    'resolution': (-20, 20),                        # Target resolution
    'group_by': 'solar_day',                        # Scene grouping
    'dask_chunks': {'x': 2048, 'y': 2048},          # Dask chunks
}

# Load data
xr_data = dc.load(**(query | load_params))

# Mask and scale (not yet generic across EASIs)
# measurement_info = dc.list_measurements().loc[query['product']]
# for vv in xr_data.data_vars:
#     scale = measurement_info.loc[vv,'scale_factor']
#     offset = measurement_info.loc[vv,'add_offset']
#     if not pd.isnull(scale) and not pd.isnull(offset):
#         xr_data[vv] = xr_data[vv] * scale + offset

# Calculate with .persist()
# TODO

display(xr_data)

In [ ]:
# Geojson (Pandas)

In [ ]:
# Web data (WMS/WCS)

## Matplotlib

Further information

- https://docs.xarray.dev/en/latest/user-guide/plotting.html

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Basic plot
# - The default 'backend' for xarray.plot is 'matplotlib'

# Select a 2D xarray grid for matplotlib
# - Xarray will label axes and title if it can
xr_data.red.isel(time=0).plot()   # The red band, first time slice, default colourbar for the data range

In [ ]:
# Subplots

xr_data.red.plot(col="time", col_wrap=4)    # The red band, all time slices one per subplot, default colourbar for the data range

In [ ]:
# Setting options

# plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# Export

## Hvplot

- Hvplot
- Holoviews
- Datashader

In [ ]:
# Holoviews, Datashader and Bokeh
import hvplot.pandas
import hvplot.xarray
import panel as pn
import colorcet as cc
import cartopy.crs as ccrs
from datashader import reductions
from holoviews import opts
# hv.extension('bokeh', logo=False)

In [ ]:
# Basic plot

In [ ]:
# Groups

In [ ]:
# Setting options

In [ ]:
# Export

## Interactive plots

- Panel
- Plotly
- Bokeh

In [ ]:
# Example: Hvplot and Panel

## Dashboards

To prototype a data driven web site and user interface.

- Streamlit